## Coletando dados de zonas e páginas do mercado livre e salvando em um dataframe

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep

In [3]:
zonas = ['sul', 'norte', 'leste', 'oeste']
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']

In [4]:
url_ml = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}' 

In [5]:
def funcao_zona(zona):
    lista_zonas = []
    lista_zonas.append(zona)

In [6]:
for zona in zonas:
    for pagina in paginas:
        funcao_zona(zona)
        print(pagina, zona)

 sul
_Desde_49 sul
_Desde_97 sul
_Desde_145 sul
_Desde_193 sul
_Desde_241 sul
 norte
_Desde_49 norte
_Desde_97 norte
_Desde_145 norte
_Desde_193 norte
_Desde_241 norte
 leste
_Desde_49 leste
_Desde_97 leste
_Desde_145 leste
_Desde_193 leste
_Desde_241 leste
 oeste
_Desde_49 oeste
_Desde_97 oeste
_Desde_145 oeste
_Desde_193 oeste
_Desde_241 oeste


In [7]:
re_quarto ='\| (.*) quarto'
re_area = '> (.*) m²'

In [8]:
def coleta_dados(url, zona):

    conteudo = requests.get(url).content
    sopa = BeautifulSoup(conteudo)
    
    precos = sopa.find_all('span', class_='price__fraction')
    areas_e_quartos = sopa.find_all('div', class_='item__attrs')
    
    lista_areas = []
    lista_quartos = []
    lista_precos = []
    lista_zonas = []
    
    for padrao in precos:
        preco = re.findall('<span class="price__fraction">(.*)</span>', str(padrao))
        lista_precos.append(preco[0].replace('.', ''))
    
    for elemento in areas_e_quartos:
        quarto = re.findall(re_quarto, str(elemento))
        area = re.findall(re_area, str(elemento))
        
        if quarto==[]:
            quarto=np.nan
        else:
            quarto=quarto[0]
        
        if area==[]:
            area=np.nan
        else:
            area=area[0]

        lista_quartos.append(quarto)
        lista_areas.append(area)
        lista_zonas.append(zona)

    dados = {'zona': lista_zonas,
             'quartos': lista_quartos,
             'area': lista_areas,
             'preco': lista_precos}

    df = pd.DataFrame(dados)  
    return df

In [9]:
lista_dfs = []
for zona in zonas:
    for pagina in paginas:
        url = url_ml.format(zona, pagina)
        df = coleta_dados(url, zona)
        print('Carregando a url', url)
        lista_dfs.append(df)
        sleep(2)
print('Dados coletados!')

Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_193
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_241
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
Carregando a url https://imoveis.mercadolivre.com.br/ca

In [10]:
dados = pd.concat(lista_dfs)

In [11]:
dados

,zona,quartos,area,preco
0,sul,1,45,950
1,sul,3,217,7000
2,sul,4,320,7000
3,sul,5,398,7000
4,sul,NaN,185,7000
...,...,...,...,...
43,oeste,2,71,1300
44,oeste,2,200,3000
45,oeste,1,40,1200
46,oeste,2,62,1300


In [12]:
dados.to_csv('dados_mercado_livre.csv', index=False)

In [13]:
df = pd.read_csv('dados_mercado_livre.csv')

In [14]:
type(dados)

pandas.core.frame.DataFrame

In [27]:
# pd.DataFrame.to_csv?

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   zona     1008 non-null   object 
 1   quartos  876 non-null    float64
 2   area     991 non-null    float64
 3   preco    1008 non-null   int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 31.6+ KB


In [16]:
df.count()

zona       1008
quartos     876
area        991
preco      1008
dtype: int64

In [17]:
df.isnull().sum()

zona         0
quartos    132
area        17
preco        0
dtype: int64

In [18]:
df.shape # linhas, colunas

(1008, 4)